In [ ]:
import numpy as np
import anndata as ad
import pandas as pd
from pandas import api
from matplotlib import interactive
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import rc_context
import scanpy as sc
sc.settings.verbosity = 3

In [ ]:
def load_gene_table(gene_table):

    if isinstance(gene_table, pd.DataFrame):
        return gene_table
    else:
        return pd.read_csv(gene_table, sep='\t').set_index('gene_id')

In [ ]:
Cellline_genes = pd.read_csv('~/PATH_TO/E-MTAB-2770-query-results.tpms.tsv', sep='\t',skiprows=4)
#https://www.ebi.ac.uk/gxa/experiments/E-MTAB-2770/

In [ ]:
Cellline_genes= Cellline_genes[['Gene ID','Gene Name','K-562, blast phase chronic myelogenous leukemia, BCR-ABL1 positive', 'THP-1, childhood acute monocytic leukemia']]

In [ ]:
K562_genes= Cellline_genes[Cellline_genes['K-562, blast phase chronic myelogenous leukemia, BCR-ABL1 positive'] > 100]
K562_genes= K562_genes[K562_genes['THP-1, childhood acute monocytic leukemia'] < 3]
K562_markerID = K562_genes['Gene Name'].tolist()

THP1_genes= Cellline_genes[Cellline_genes['THP-1, childhood acute monocytic leukemia'] > 200]
THP1_genes=  THP1_genes[THP1_genes['K-562, blast phase chronic myelogenous leukemia, BCR-ABL1 positive'] < 5]
THP1_markerID = THP1_genes['Gene Name'].tolist()

print(len(THP1_markerID), len(K562_markerID))

In [ ]:
ref_ID= K562_markerID + THP1_markerID

In [ ]:
adata = sc.read_umi_tools('~/PATH_TO/processed/cDNA/cDNA_fk209_2022-05-24-10X_re.star.gene_cell_counts.txt.gz')

In [ ]:
var_names = adata.var_names.intersection(adata.var_names)
adata = adata[:, var_names]

In [ ]:
adata

In [ ]:
gene_table = load_gene_table('~/PATH_TO/gencode.v38.annotation.txt')
for col in gene_table.columns:
   adata.var.loc[:, col] = gene_table[col]

In [ ]:
for col in gene_table.columns:
   adata.var.loc[:, col] = gene_table[col]

In [ ]:
def K562(adata):
    #adata = self.adata
    K562_markers = K562_genes['Gene Name'].tolist()
    K562_genes_mm_ens = adata.var_names[np.in1d(adata.var.gene_name, K562_markers)]
    sc.tl.score_genes(adata, K562_genes_mm_ens, score_name='K562_score')

In [ ]:
def THP1(adata):
    #adata = self.adata
    THP1_markers = THP1_genes['Gene Name'].tolist()
    THP1_genes_mm_ens = adata.var_names[np.in1d(adata.var.gene_name, THP1_markers)]
    sc.tl.score_genes(adata, THP1_genes_mm_ens, score_name='THP1_score')

In [ ]:
#gene全体を使うときのみ
sc.pp.filter_cells(adata, min_genes=250)
sc.pp.filter_genes(adata, min_cells=25)

In [ ]:
adata

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
adata.var['mt'] = adata.var.seqname == 'chrM'  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.1, multi_panel=True)

In [ ]:
adata = adata[adata.obs.pct_counts_mt < 8, :] 
adata = adata[adata.obs.pct_counts_mt > 2, :] 
adata = adata[adata.obs.total_counts < 8000, :]
adata = adata[adata.obs.total_counts > 1000, :]
adata = adata[adata.obs.n_genes_by_counts < 3500,:]

In [ ]:
K562(adata)
THP1(adata)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.1, multi_panel=True)

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=8e3)
sc.pp.log1p(adata)
adata_raw = adata

In [ ]:
#adata_raw = adata
adata = adata_raw

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.025, max_mean=2, min_disp=0.5)
#sc.pp.highly_variable_genes(adata)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.var['ref_ID'] = np.in1d(adata.var.gene_name, ref_ID)
adata

In [ ]:
adata = adata[:, adata.var.ref_ID]
adata

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color=adata.obs, projection='2d', size=100)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=7)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=200, n_pcs=5)
sc.tl.umap(adata)
sc.pl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.1)

In [ ]:
with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(adata, color=adata.obs)

In [ ]:
plt.set_cmap('viridis')
plt.cm.get_cmap()

In [ ]:
for col in adata.obs:
    with rc_context({'figure.figsize': (5, 5)}):
        sc.pl.umap(adata, color=col, size=10, save="_fk209_clusteing"+str(col)+".pdf")

In [ ]:
adata.write_h5ad('~/PATH_TO/processed/others/2022-05-24-10X_final/2022-05-24-10X/fk209_proc_cDNA.h5ad')